In [1]:
import torch
import pandas as pd
from torch.utils.data import Dataset

In [2]:
PATH = "C:/Projects/TFM/dataset/AD_MCI_HC_WINDOWED"
INDEX_PATH = "C:/Projects/TFM/dataset/AD_MCI_HC_WINDOWED/data.csv"

In [3]:
from entities.graphs.graph_builder import RawAndPearson, MomentsAndPearson, MomentsAndPLI, RawAndPLI, PSDAndCSD, PSDAndPearson, OfflineGeneric
from entities.graphs.data_reader import read_record

In [4]:
class BaseDataset(Dataset):
    def __init__(self, indices, builder, transform=None, target_transform=None):
        self.indices = indices
        self.builder = builder
        self.transform = transform
        self.target_transform = target_transform
        

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        current_path = self.indices.iloc[idx]["path"]
        raw_data = read_record(current_path)
        label = self.indices.iloc[idx]["label"]
        data = self.builder.build(raw_data, label)
        return data
    
class OfflineDataset(Dataset):
    def __init__(self, node_indices, edge_indices, builder, transform=None, target_transform=None):
        self.node_indices = node_indices
        self.edge_indices = edge_indices
        self.builder = builder
        self.transform = transform
        self.target_transform = target_transform
        

    def __len__(self):
        return len(self.node_indices)

    def __getitem__(self, idx):
        current_path_nodes = self.node_indices.iloc[idx]["path"]
        computed_nodes = read_record(current_path_nodes)
        
        current_path_edges = self.edge_indices.iloc[idx]["path"]
        computed_edges = read_record(current_path_edges)
        
        label = self.node_indices.iloc[idx]["label"]
        data = self.builder.build(computed_nodes, computed_edges, label)
        
        return data

In [5]:
from sklearn.model_selection import train_test_split

MODE = "OFFLINE"

if MODE == "OFFLINE":

    NODE_INDEX_PATH = "C:/Projects/TFM/dataset/AD_MCI_HC_PSD/data.csv"
    EDGE_INDEX_PATH = "C:/Projects/TFM/dataset/AD_MCI_HC_PEARSON/data.csv"

    node_indices = pd.read_csv(NODE_INDEX_PATH, index_col="Unnamed: 0")
    edge_indices = pd.read_csv(EDGE_INDEX_PATH, index_col="Unnamed: 0")

    node_indices = node_indices.drop(node_indices[node_indices.label == "MCI"].index)
    node_indices_hc = node_indices[node_indices.label == 'HC'].sample(frac=0.4)
    node_indices_ad = node_indices[node_indices.label == 'AD']
    node_indices = pd.concat([node_indices_hc, node_indices_ad])

    node_train_indices, node_test_indices = train_test_split(node_indices, shuffle=True)
    edge_train_indices, edge_test_indices = edge_indices.iloc[node_train_indices.index], edge_indices.iloc[node_test_indices.index]

    builder = OfflineGeneric(th=None)
    
    train_dataset = OfflineDataset(node_train_indices, edge_train_indices, builder)
    test_dataset = OfflineDataset(node_test_indices, edge_test_indices, builder)
    
if MODE == "ONLINE":
    indices = pd.read_csv(INDEX_PATH, index_col="Unnamed: 0")
    indices = indices.drop(indices[indices.label == "MCI"].index)
    indices_hc = indices[indices.label == 'HC'].sample(frac=0.4)
    indices_ad = indices[indices.label == 'AD']
    indices = pd.concat([indices_hc, indices_ad])

    train_data, test_data = train_test_split(indices, shuffle=True)

    builder = RawAndPearson(normalize_nodes=True, normalize_edges=False, th=0)
    #builder = MomentsAndPearson(th=0)
    #builder = MomentsAndPLI()
    #builder = RawAndPLI(normalize_nodes=True, normalize_edges=False)
    #builder = PSDAndCSD()
    #builder = PSDAndPearson(th=0.5)

    train_dataset = BaseDataset(train_data, builder)
    test_dataset = BaseDataset(test_data, builder)
    train_data

In [6]:
from torch_geometric.loader import DataLoader

_BATCH_SIZE = 128
train_dataloader = DataLoader(train_dataset, batch_size=_BATCH_SIZE, shuffle=True)#sampler=weighted_sampler)
test_dataloader = DataLoader(test_dataset, batch_size=_BATCH_SIZE, shuffle=True)

In [7]:
def check_data():
    for data in train_dataloader:
        print(data.edge_attr, data.x.shape)
        print(next(iter(data[0]))[1].shape)
        break

check_data()

    

tensor([[1.0000, 0.8407, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.8407, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.7872],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.7628],
        [0.0000, 0.0000, 0.0000,  ..., 0.7872, 0.7628, 0.0000]],
       dtype=torch.float64) torch.Size([2432, 6])
torch.Size([19, 6])


In [8]:
from entities.models.factory import ModelFactory
from entities.models.modelsTypes import Model

model_factory = ModelFactory()
model = model_factory.create(Model.EEGCONVNETMINIV2)
model = model.double()

In [9]:
model

EEGConvNetMiniV2(
  (conv1): GCNConv(-1, 32)
  (conv2): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [10]:
criterion = torch.nn.CrossEntropyLoss()#weight=torch.tensor([3, 1], dtype=torch.float64)) 
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', verbose=True, patience=5, threshold=0.05, cooldown=2, factor=0.5, min_lr=1e-5)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.1, verbose=True)

In [11]:
from sklearn.metrics import roc_auc_score, balanced_accuracy_score
from numpy import mean
import numpy as np

auroc_train_history = []
auroc_test_history = []
balACC_train_history = []

balACC_test_history = []
loss_train_history = []
loss_test_history = []

train_accs = []
test_accs = []

_NUM_EPOCHS = 120
_DEVICE = torch.device("cpu")


def train():
    model.train()
    running_loss = 0.0
    batch_loss = []
    mean_batch_loss = 0
    for i, data in enumerate(train_dataloader):  # Iterate in batches over the training dataset.

        #data.batch = data.batch.view(data.batch.shape[0], -1)
        #print(data.x[0].shape)
        optimizer.zero_grad()  # Clear gradients.
        out = model(data.x, data.edge_index, data.edge_attr,
                    data.batch)  # Perform a single forward pass.
        
        

        loss = criterion(out, data.label)  # Compute the loss.
        #print(loss.item())
        batch_loss.append(loss.item())
        #print(data.label)
        loss.backward()  # Derive gradients.
          # Update parameters based on gradients.
        optimizer.step()
        
        running_loss += loss.item()
        if i%30 == 29:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 30:.3f}')
            running_loss = 0.0
            
    mean_batch_loss = mean(batch_loss)
    print(f"Training epoch {epoch}: {mean_batch_loss:.3f} loss")
    scheduler.step(mean_batch_loss)

def test(loader):
    model.eval()

    correct = 0
    y_probs_train = torch.empty(0, 2).to(_DEVICE)
    y_true_train, y_pred_train = [], []
    with torch.no_grad():
        for data in loader:  # Iterate in batches over the training/test dataset.
            out = model(data.x, data.edge_index, data.edge_attr, data.batch)
            y_batch = data.label.to(device=_DEVICE, non_blocking=True)
            
            pred = out.argmax(dim=1)  # Use the class with highest probability.
            
            #print(pred)
            
            correct += int((pred == data.label).sum())  # Check against ground-truth labels.
            y_pred_train += pred.cpu().numpy().tolist()
            
            y_probs_train = torch.cat((y_probs_train, out.data), 0)
            y_true_train += y_batch.cpu().numpy().tolist()
            
    y_probs_train = torch.nn.functional.softmax(y_probs_train, dim=1).cpu().numpy()
    y_true_train = np.array(y_true_train)

    return correct / len(loader.dataset), y_true_train, y_probs_train, y_pred_train  # Derive ratio of correct predictions.

best_model_test_acc = 0
for epoch in range(300):
    train()
    
    train_acc, y_true_train, y_probs_train, y_pred_train = test(train_dataloader)
    test_acc, y_true_test, y_probs_test, y_pred_test = test(test_dataloader)
    train_accs.append(train_acc)
    test_accs.append(test_acc)

    
    balACC_train_history.append(balanced_accuracy_score(y_true_train, y_pred_train))
    balACC_test_history.append(balanced_accuracy_score(y_true_test, y_pred_test))

    print(f"Train Bal.ACC: {balACC_train_history[-1]:.3f}, test Bal.ACC: {balACC_test_history[-1]:.3f}")
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
    
    if test_acc > best_model_test_acc: torch.save(model.state_dict(), './trained_models/eegconvnetminiv2-128bs-PSDAndPearson-noTH.pt')
    #scheduler.step(test_acc)

[1,    30] loss: 1.002
Training epoch 0: 0.980 loss
Train Bal.ACC: 0.625, test Bal.ACC: 0.621
Epoch: 000, Train Acc: 0.6160, Test Acc: 0.6202
[2,    30] loss: 0.614
Training epoch 1: 0.611 loss
Train Bal.ACC: 0.649, test Bal.ACC: 0.656
Epoch: 001, Train Acc: 0.6382, Test Acc: 0.6547
[3,    30] loss: 0.558
Training epoch 2: 0.561 loss
Train Bal.ACC: 0.722, test Bal.ACC: 0.717
Epoch: 002, Train Acc: 0.7238, Test Acc: 0.7176
[4,    30] loss: 0.548


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_true_test, y_pred_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=["AD", "HC"])

disp.plot()

# Experiments

In [ ]:
PATH = "C:/Projects/TFM/dataset/AD_MCI_HC_WINDOWED"
INDEX_PATH = "C:/Projects/TFM/dataset/AD_MCI_HC_WINDOWED/data.csv"

In [ ]:
import pandas as pd
import numpy as np
from entities.graphs.data_reader import read_record

In [ ]:
df = pd.read_csv(INDEX_PATH)
dataPath = df.iloc[828].path
dataPath2 = df.iloc[20].path
record = read_record(dataPath)
record2 = read_record(dataPath2)
#df[df["path"].str.contains("Patient_2_AD_T0_filtered_0")]

In [ ]:
from entities.graphs.node_extractors import PSDExtractor
from entities.graphs.edge_extractors import SpectralCoherenceExtractor
from scipy.signal import csd

e = SpectralCoherenceExtractor()
xd = e.extract_features(record2)
xd.shape

#a = [[np.abs(np.mean(csd(ch1, ch2, fs=256)[1])) for ch2 in record] for ch1 in record]
#a

## Trying PLI implementation using Hilbert Transform

In [ ]:
from scipy.signal import hilbert
import numpy as np


hilbert_transform = np.imag(hilbert(record, axis=1))
phase_diff = np.zeros((hilbert_transform.shape[0], hilbert_transform.shape[0]))

for row in range(19):
    for col in range(19):
        phase_diff[row, col] = np.abs(np.mean(np.exp(1j*(hilbert_transform[row] - hilbert_transform[col]))))

phase_diff
#hilbert_transform

In [ ]:
def extract_features(data, normalize=False, th=None) -> None:

        rows, cols = data.shape
        PLImatrix = np.zeros((rows, rows))    
        
        hilbert_transform = np.imag(hilbert(data))
        
        if normalize:
            data = normalize(data)
            
        for row in range(rows):
            for col in range(rows):
                PLImatrix[row, col] = np.abs(np.mean(np.exp(1j * (hilbert_transform[row] - hilbert_transform[col]))))
          
        # Set to 0 values undes a th
        if th:
            PLImatrix = np.where(
                abs(PLImatrix) < th, 0, PLImatrix
            )
        
        # Remove self loops
        PLImatrix = np.where(
                abs(PLImatrix) == 1.0, 0, PLImatrix
            )
        
        PLImatrix = torch.from_numpy(PLImatrix)
        return PLImatrix

extract_features(record)

In [ ]:
empt = np.ndarray((2, 19, 1280))
empt[0] = record
empt[1] = record2
a = spectral_connectivity_epochs(empt, method="pli", sfreq=256, mode="multitaper")
data = a.get_data()

In [ ]:
from scipy.signal import csd

def expectation(data):
    vals = np.unique(data)
    probs = [
        np.count_nonzero(data == val) / len(data) for val in vals
    ]
    return np.sum(vals * probs)

In [ ]:
f, sxy = csd(record[0], record[2], fs=256)
expectation(np.sign(np.imag(sxy)))

In [ ]:
def _compute_pli(x, y):
    _, cpsd = csd(x, y, fs=256)
    phase_lag_index = np.mean(np.sign(np.imag(cpsd[30:60])))
    return phase_lag_index

In [ ]:

rows, cols = record.shape
PLImatrix = np.zeros((rows, rows))    
    
    
for row in range(rows):
    for col in range(rows):
        PLImatrix[row, col] = _compute_pli(record[row], record[col])


#PLImatrix = torch.from_numpy(PLImatrix)
PLImatrix

In [ ]:
from spectral_connectivity import Multitaper, Connectivity

# Compute multitaper spectral estimate
signal = np.zeros((1280, 19))
signal = np.reshape(record, (1280, 1, 19))
m = Multitaper(time_series=signal,
               sampling_frequency=256,
               start_time=0)

# Sets up computing connectivity measures/power from multitaper spectral estimate
c = Connectivity.from_multitaper(m)

# Here are a couple of examples
power = c.power() # spectral power
coherence = c.coherence_magnitude()
phase_lag_index = c.phase_lag_index()
#canonical_coherence = c.canonical_coherence(brain_area_labels)

In [ ]:
print(list(c.frequencies).index(10.4))

phase_lag_index[0][10]